# Sample Skinning Example without higherarchical bones

In the following skinning example, we have made the assumption that all offset matrices are identity matrices. This is a simplification that allows us to focus on the skinning process itself. In practice, the offset matrices are used to transform the vertices from the bone space to the model space.

In [ ]:

from Elements.definitions import MODEL_DIR, PICKLES_DIR
import Elements.pyECSS.math_utilities as util
from os import path
import time 
import pickle
import numpy as np
import meshplot as mp

In [ ]:

def vec9_to_transformation_matrix(vec9):
    # takes a vector of 9 parameters and returns a transformation matrix
    return util.translate(vec9[0:3])@util.eulerAnglesToRotationMatrix(vec9[3:6]) @ util.scale(vec9[6:9])

def vertex_apply_M(v,M):
    # applies a transformation matrix to a vertex
    # v is a 3x1 numpy array, e.g., [1,2,3]
    # M is a 4x4 numpy array; a standard transformation matrix
    # the output is a 3x1 numpy array e.g., [2,3,4]
    return np.dot(M,np.append(v,[1.0]))[0:3]

# Loading the model 

In [ ]:

vertices = np.array([ [0,0,0], #0
                      [1,0,0], #1
                      [0,1,0], #2
                      [1,1,0], #3
                      [0,2,0], #4
                      [1,2,0], #5
                      ])
faces = np.array([ [0,1,2],[2,1,3], [2,3,4], [4,3,5] ])
p = mp.plot(vertices, faces,vertices[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  

# we are not using hierarchical bones here
# we simply have 3 bones 
# in the following list we write down the weights/influence for each vertex:
# for example 
# vertex 0 is only influenced by bone 0
# vertex 2 is influenced by bone 0 and 1 with equal weight 1/2
# vertex 4 is influenced by bone 1 and 2 with equal weight 1/2
weights = np.array([ [1,0,0], #0
                     [1,0,0], #1
                     [1/2,1/2,0], #2 
                     [1/2,1/2,0], #3
                     [0,1/2,1/2], #4
                     [0,1/2,1/2], #5
                     ])




# Visualizing the T-pose of the model

In [ ]:


# play around with these values
# the first 3 are translation, the next 3 are rotation, the last 3 are scale
jO = [0.0, 0, 0, 0, 0, 0, 1, 1, 1]
j1 = [0.0, 0, 0, 1, 1, 0, 1, 1, 1]
j2 = [0.0, 0, 0, 0, 0, 0, 1, 1, 1]

# these are the transformation matrices for each bone
BB = np.array([vec9_to_transformation_matrix(jO),
               vec9_to_transformation_matrix(j1),
               vec9_to_transformation_matrix(j2)
               ])

newv = np.zeros([len(vertices),3])
print(newv)
start = time.time()
for i in range(len(vertices)):  
    for j in range(3): # we only have 3 bones
        newv[i] = newv[i] + weights[i,j]*vertex_apply_M(vertices[i],BB[j])
end = time.time()

p = mp.plot(newv, faces,newv[:, 1],shading={"scale": 2.5,"wireframe":True},return_plot=True)  
p.add_mesh(vertices+[2,0,0], faces, vertices[:,1], shading={"scale": 2.5,"wireframe":True})